In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import h5py
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
%matplotlib inline
from resnet import *
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import time
import math
from utils import AverageMeter
import argparse

In [2]:
def parse_args():
    parser = argparse.ArgumentParser(description='alzheimer desease recognnition')
    parser.add_argument('--train_images', default='/data/beast/df/alzheimer/train.h5')
    parser.add_argument('--train_labels', default='/data/beast/df/alzheimer/train.csv')
    parser.add_argument('--val_images', default='/data/beast/df/alzheimer/val.h5')
    parser.add_argument('--val_labels', default='/data/beast/df/alzheimer/val.csv')
    parser.add_argument('--out_dir', default='/data/beast/df/alzheimer/out_model')
    parser.add_argument('--phase', default='train', choices=['train', 'val', 'test', 'predict'])
    parser.add_argument('--model', default='resnet34')
    parser.add_argument('--weight', default=None)
    parser.add_argument('--lr', default=1e-3, type=float)
    parser.add_argument('--mom', default=0.9, type=float)
    parser.add_argument('--wd', default=1e-4, type=float)
    parser.add_argument('--fix', default=50, type=int)
    parser.add_argument('--step', default=20, type=int)
    parser.add_argument('--epoch', default=100, type=int)
    parser.add_argument('--display', default=8, type=int)
    parser.add_argument('--num_workers', default=2, type=int)
    parser.add_argument('--batch_size', default=32, type=int)
    return parser.parse_args(args=[])

opt = parse_args()

In [3]:
class AlzheimerDS(Dataset):
    def __init__(self, infile, label_file):
        f = h5py.File(infile)
        images = f['data']
        self.images = np.transpose(images, [0,1,3,2,4])
        self.mean = np.mean(images)
        self.std = np.std(images)
        self.images = (self.images - self.mean)/self.std
        self.images = torch.from_numpy(self.images).float()
        df_labels = pd.read_csv(label_file)
        self.labels = list(df_labels['label'])
    def __getitem__(self, item):
        image = self.images[item]
        return image, self.labels[item]
    def __len__(self):
        return self.images.shape[0]

In [4]:
# ds = AlzheimerDS('/data/beast/df/alzheimer/train_pre_data.h5', 
#                 '/data/beast/df/alzheimer/train_pre_label.csv')

In [5]:
# train_dataloader = DataLoader(ds, batch_size=4, shuffle=True, num_workers=2)

In [6]:
# model = resnet34(num_classes=3, shortcut_type=True, sample_size=79, sample_duration=95)

In [7]:
def initial_cls_weights(cls):
    for m in cls.modules():
        if isinstance(m, nn.Conv2d):
            n = m.kernel_size[0]*m.kernel_size[1]*m.out_channels
            m.weight.data.normal_(0, math.sqrt(2./n))
            if m.bias is not None:
                m.bias.data.zero_()
        if isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()
        if isinstance(m, nn.Conv3d):
            n = m.kernel_size[0]*m.kernel_size[1]*m.kernel_size[2]*m.out_channels
            m.weight.data.normal_(0, math.sqrt(2./n))
            if m.bias is not None:
                m.bias.data.zero_()

In [8]:
# initial_cls_weights(model)

In [9]:
# model_cuda = nn.DataParallel(model).cuda()
# criterion = nn.CrossEntropyLoss().cuda()

In [10]:
# lr = 1e-3
# mom = 0.9
# wd = 1e-4

# optimizer = optim.SGD([
#     {'params': model.parameters()}
# ], lr=lr, momentum=mom, weight_decay=wd, nesterov=True)

In [11]:
def train(train_data_loader, model, criterion, optimizer, epoch, display):
    model.train()
    tot_pred = np.array([], dtype=int)
    tot_label = np.array([], dtype=int)
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()
    end = time.time()
    logger = []
    for num_iter, (images, labels) in enumerate(train_data_loader):
        data_time.update(time.time()-end)
#         print(labels)
        output = model(Variable(images.cuda()))
        loss = criterion(output, Variable(labels.cuda()))
#         print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_time.update(time.time()-end)
        _,pred = torch.max(output, 1)
        pred = pred.cpu().data.numpy().squeeze()
        labels = labels.numpy().squeeze()
        tot_pred = np.append(tot_pred, pred)
        tot_label = np.append(tot_label, labels)
        losses.update(loss.data.cpu().numpy(), len(images))
        accuracy.update(np.equal(pred, labels).sum()/len(labels), len(labels))
        end = time.time()
        if (num_iter+1) % display == 0:
            correct = np.equal(tot_pred, tot_label).sum()/len(tot_pred)
            print_info = 'Epoch: [{0}][{1}/{2}]\tTime {batch_time.val:3f} ({batch_time.avg:.3f})\t'\
                'Data {data_time.avg:.3f}\t''Loss {loss.avg:.4f}\tAccuray {accuracy.avg:.4f}'.format(
                epoch, num_iter, len(train_data_loader),batch_time=batch_time, data_time=data_time,
                loss=losses, accuracy=accuracy
            )
            print(print_info)
            logger.append(print_info)
    return accuracy.avg, logger


def val(train_data_loader, model, criterion, optimizer, epoch, display):
    model.train()
    tot_pred = np.array([], dtype=int)
    tot_label = np.array([], dtype=int)
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    accuracy = AverageMeter()
    end = time.time()
    logger = []
    for num_iter, (images, labels) in enumerate(train_data_loader):
        data_time.update(time.time()-end)
#         print(labels)
        output = model(Variable(images.cuda()))
        loss = criterion(output, Variable(labels.cuda()))
#         print(loss)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
        batch_time.update(time.time()-end)
        _,pred = torch.max(output, 1)
        pred = pred.cpu().data.numpy().squeeze()
        labels = labels.numpy().squeeze()
        tot_pred = np.append(tot_pred, pred)
        tot_label = np.append(tot_label, labels)
        losses.update(loss.data.cpu().numpy(), len(images))
        accuracy.update(np.equal(pred, labels).sum()/len(labels), len(labels))
        end = time.time()
        if (num_iter+1) % display == 0:
            correct = np.equal(tot_pred, tot_label).sum()/len(tot_pred)
            print_info = 'Epoch: [{0}][{1}/{2}]\tTime {batch_time.val:3f} ({batch_time.avg:.3f})\t'\
                'Data {data_time.avg:.3f}\t''Loss {loss.avg:.4f}\tAccuray {accuracy.avg:.4f}'.format(
                epoch, num_iter, len(train_data_loader),batch_time=batch_time, data_time=data_time,
                loss=losses, accuracy=accuracy
            )
            print(print_info)
            logger.append(print_info)
    return accuracy.avg, logger

In [12]:
# for i in range(10):
#     train(train_dataloader, nn.DataParallel(model).cuda(), criterion, optimizer, i, 70)

In [13]:
def main():
    print('====> parse options')
    opt = parse_args()
    print(opt)
    
    os.makedirs(opt.out_dir, exist_ok=True)
    time_stamp = time.strftime('%Y%m%d%H%M%S', time.localtime(time.time()))
    out_dir = os.path.join(opt.out_dir, 'alzheimer_recognition_{}'.format(time_stamp))
    os.makedirs(out_dir, exist_ok=True)
    print('====> building model:')
    model = resnet34(num_classes=3, shortcut_type=True, sample_size=79, sample_duration=95)
    initial_cls_weights(model)
    criterion = nn.CrossEntropyLoss().cuda()
    if opt.phase == 'train':
#         train_ds = AlzheimerDS(opt.train_images, opt.train_labels)
#         train_dataloader = DataLoader(train_ds, batch_size=opt.batch_size, shuffle=True, 
#                                       num_workers=opt.num_workers, pin_memory=True)
        train_ds = AlzheimerDS(opt.train_images, opt.train_labels)
        train_dataloader = DataLoader(train_ds, batch_size=opt.batch_size, shuffle=True, 
                                      num_workers=opt.num_workers, pin_memory=True)
        val_ds = AlzheimerDS(opt.val_images, opt.val_labels)
        val_dataloader = DataLoader(val_ds, batch_size=opt.batch_size, shuffle=True, 
                                      num_workers=opt.num_workers, pin_memory=True)
        beat_acc = 0.7
        for epoch in range(opt.epoch):
            if epoch < opt.fix:
                lr = opt.lr
            else:
                lr = opt.lr * (0.1 ** (epoch//opt.step))
            optimizer = None
            optimizer = optim.SGD([{'params': model.parameters()}], 
                                  lr=opt.lr, momentum=opt.mom, weight_decay=opt.wd, nesterov=True)
            
            _, _ = train(train_dataloader, nn.DataParallel(model).cuda(), criterion, optimizer, epoch, opt.display)
            acc, logger = val(val_dataloader, nn.DataParallel(model).cuda(), criterion, optimizer, epoch, opt.display)
            if acc > beat_acc:
                print('\ncurrent best accuracy is: {}\n'.format(acc))
                beat_acc = acc
                saved_model_name = os.path.join(out_dir, 'alzheimer_recognition_{:04d}_best.pth'.format(epoch))
                torch.save(model.cpu().state_dict(), saved_model_name)
                print('====> save model:\t{}'.format(saved_model_name))
                
    
                
                

In [14]:
main()

====> parse options
Namespace(batch_size=32, display=8, epoch=100, fix=50, lr=0.001, model='resnet34', mom=0.9, num_workers=2, out_dir='/data/beast/df/alzheimer/out_model', phase='train', step=20, train_images='/data/beast/df/alzheimer/train.h5', train_labels='/data/beast/df/alzheimer/train.csv', val_images='/data/beast/df/alzheimer/val.h5', val_labels='/data/beast/df/alzheimer/val.csv', wd=0.0001, weight=None)
====> building model:


/home/ubuntu/beast/code/DFSolution/alzheimer/resnet.py:145: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


RuntimeError: CUDA out of memory. Tried to allocate 150.00 MiB (GPU 0; 31.72 GiB total capacity; 3.51 GiB already allocated; 24.56 MiB free; 739.50 KiB cached)

In [ ]:
# print('====> parse options')
# opt = parse_args()
# print(opt)
    
#     os.makedirs(opt.out_dir, exist_ok=True)
#     time_stamp = time.strftime('%Y%m%d%H%M%S', time.localtime(time.time()))
#     out_dir = os.path.join(opt.out_dir, 'alzheimer_recognition_{}'.format(time_stamp))
#     os.makedirs(out_dir, exist_ok=True)
#     print('====> building model:')
# model = resnet34(num_classes=3, shortcut_type=True, sample_size=79, sample_duration=95)
# initial_cls_weights(model)

In [ ]:
ls '/data/beast/df/alzheimer/out_model/alzheimer_recognition_20191018150905/'

In [ ]:
model_weights = '/data/beast/df/alzheimer/out_model/alzheimer_recognition_20191018150905/alzheimer_recognition_0046_best.pth'
model.load_state_dict(torch.load(model_weights))

In [ ]:
def calc_stat_info(infile):
    f = h5py.File(infile)
    images = f['data']
    mean = np.mean(images)
    std = np.std(images)
    return mean, std

mean, std = calc_stat_info(opt.train_images)
print('mean:\t{}'.format(mean))
print('std:\t{}'.format(std))

In [ ]:
class AlzheimerPredictDS(Dataset):
    def __init__(self, infile, mean, std):
        f = h5py.File(infile)
        images = f['data']
        self.images = np.transpose(images, [0,1,3,2,4])
        self.mean = mean
        self.std = std
        self.images = (self.images - self.mean)/self.std
        self.images = torch.from_numpy(self.images).float()
    def __getitem__(self, item):
        image = self.images[item]
        return image, item
    def __len__(self):
        return self.images.shape[0]
    
# eval_ds = AlzheimerPredictDS('/data/beast/df/alzheimer/testa.h5', mean, std)
# eval_data_loader = DataLoader(eval_ds, batch_size=4, num_workers=2, shuffle=False, pin_memory=False)
# for i, (images, items) in enumerate(eval_data_loader):
#     print(images.shape)
#     print(items)

In [ ]:
def predict_eval(eval_data_loader, model, display):
    model.eval()
    tot_pred = np.array([], dtype=int)
    tot_items = np.array([], dtype=int)
    batch_time = AverageMeter()
    data_time = AverageMeter()
    end = time.time()
    logger = []
    for num_iter, (images, items) in enumerate(eval_data_loader):
        data_time.update(time.time()-end)
        output = model(Variable(images.cuda()))
        _,pred = torch.max(output, 1)
        pred = pred.cpu().data.numpy().squeeze()
        items = items.numpy().squeeze()
        batch_time.update(time.time()-end)
        tot_pred = np.append(tot_pred, pred)
        tot_items = np.append(tot_items, items)
        end = time.time()
    return tot_pred, tot_items
        
# pred, items = predict_eval(eval_data_loader, nn.DataParallel(model).cuda(), 2)
# print(pred)
# print(items)

In [ ]:
def predict(infile, model_weights=None):
    print('====> parse options')
    opt = parse_args()
    print(opt)
    model = resnet34(num_classes=3, shortcut_type=True, sample_size=79, sample_duration=95)
    initial_cls_weights(model)
    model_weights = '/data/beast/df/alzheimer/out_model/alzheimer_recognition_20191018150905/alzheimer_recognition_0085_best.pth'
#     if not model_weights:
    model.load_state_dict(torch.load(model_weights))
    eval_ds = AlzheimerPredictDS(infile, mean, std)
    eval_data_loader = DataLoader(eval_ds, batch_size=4, num_workers=2, shuffle=False, pin_memory=False)
    pred, items = predict_eval(eval_data_loader, nn.DataParallel(model).cuda(), 2)
    return pred, items

In [ ]:
infile_a = '/data/beast/df/alzheimer/testa.h5'
infile_b = '/data/beast/df/alzheimer/testb.h5'
pred_a, items_a = predict(infile_a)
pred_b, items_b = predict(infile_b)
test_id = ['testa_{}'.format(i) for i in items_a] + ['testb_{}'.format(i) for i in items_b]
test_id = np.array(test_id)
label = np.array([], dtype=int)
label = np.append(label, pred_a)
label = np.append(label, pred_b)
print(label)
print(test_id)

In [ ]:
d = {'testa_id': test_id, 'label': label}
df_result = pd.DataFrame(data=d)
df_result.to_csv('./result.csv', index=False)